# P1 - Identificando personas desde una imagen de su cara

Elaborado por GRUPO 1:

- Juanita Piraban Barbosa - 201216313
- Lorena Morales Rodríguez - 202027957
- Alejandro Barinas Guio - 201628859
- Jaime Humberto Trujillo Perea - 201920366
- Alexander Zapata Galindo - 201425426

In [90]:
from tensorflow.python.keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,ZeroPadding2D,Dropout,Softmax
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.python.keras import Sequential
from sklearn.datasets import fetch_lfw_people
from sklearn.datasets import fetch_lfw_pairs
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
import scikitplot as skplt
import numpy as np
import random
import tensorflow_datasets as tfds
import tensorflow as tf

from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_lfw_people
from keras import backend as K
from livelossplot import PlotLossesKeras
# others
from random import randrange
from time import time
# hyperparameter optimization
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [91]:
# DataSet : “Labeled Faces in the Wild”
min_faces_per_person = 60 # número mínimo de fotos por personaje para filtrar la base
resize=0.5 # Número de define la calidad de las imágenes utilizar 0.5 y 0.7

# con 10 mín  fotos se generan 158 categorías
# con 20 mín  fotos se generan 62 categorías
# con 40 mín  fotos se generan 19 categorías
# con 50 mín  fotos se generan 12 categorías
# Probar y comparar 20, 40 y 70 

In [92]:
# Some definitions

data = fetch_lfw_people(min_faces_per_person=min_faces_per_person, resize=resize)

l=list(data.keys())
X=data[l[0]]
y=data[l[2]]
target_names=data[l[3]]

_,W,H=data[l[1]].shape
features=X.shape[1]
m=X.shape[0]
classes=data[l[3]].shape[0]

print("Resumen Imágenes a procesar")
print("Samples:",m)
print("Classes:",classes)
print("Dimentions:",(W,H))

Resumen Imágenes a procesar
Samples: 1348
Classes: 8
Dimentions: (62, 47)


In [93]:
# Shuffling the data

temp=list(zip(X,y)) 
random.shuffle(temp) 
X,y=zip(*temp)

X=np.array(X)
Y=np.array(y)

In [94]:
# Train and test splitting of data
training_data_X,testing_data_X,training_data_Y,testing_data_Y=train_test_split(X,y,test_size=0.20,random_state=42)

In [95]:
# Categorizar y redimensionar bases
training_data_Y=to_categorical(training_data_Y)
testing_data_Y=to_categorical(testing_data_Y)

training_data_X=training_data_X.reshape(training_data_X.shape[0],W,H,1)
testing_data_X=testing_data_X.reshape(testing_data_X.shape[0],W,H,1)

### MODELO 5 - CNN

In [57]:
# CNN model
K.clear_session()
model=Sequential()

model.add(Conv2D(64,kernel_size=3,activation='relu',input_shape=(W,H,1)))
model.add(MaxPooling2D((2,2),strides=(2,2)))
model.add(Conv2D(128,kernel_size=3,activation='relu'))
model.add(MaxPooling2D((2,2),strides=(2,2)))
model.add(Conv2D(256,kernel_size=5,activation='relu'))
model.add(MaxPooling2D((2,2),strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(classes,activation='softmax'))
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

In [58]:
# some parameters
size = 128
epoc = 50 # Número de épocas


checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
# fit model

model.fit(training_data_X,training_data_Y,validation_data=(testing_data_X,testing_data_Y),batch_size=size,\
          epochs=epoc, verbose=1, callbacks=[PlotLossesKeras()])

In [ ]:
# model's performance on test set
test_loss, test_acc = model.evaluate(testing_data_X,testing_data_Y, verbose=1)

### HYPERPARAMETER - CNN

In [126]:
# function to create the model for Keras wrapper to scikit learn
# we will optimize the type of pooling layer (max or average) and the activation function of the 2nd and 3rd convolution layers 
def create_cnn_model(pool_type='max', conv_activation='sigmoid', dropout_rate=0.10, optimizer = 'adam'):
    # create model
    # CNN model
    K.clear_session()
    model = Sequential()

    # first layer: convolution
    model.add(Conv2D(64,kernel_size=3,activation='relu',input_shape=(W,H,1)))
    if pool_type == 'max':
        model.add(MaxPooling2D(pool_size=(2, 2)))
    if pool_type == 'average':
        model.add(AveragePooling2D(pool_size=(2, 2)))
    
    # second series of layers: convolution, pooling
    model.add(Conv2D(128,kernel_size=5,activation='relu'))
    if pool_type == 'max':
        model.add(MaxPooling2D(pool_size=(2, 2)))
    if pool_type == 'average':
        model.add(AveragePooling2D(pool_size=(2, 2)))

    # third series of layers: convolution, pooling, and dropout    
    model.add(Conv2D(256,kernel_size=5,activation='relu'))
    if pool_type == 'max':
        model.add(MaxPooling2D(pool_size=(2, 2)))
    if pool_type == 'average':
        model.add(AveragePooling2D(pool_size=(2, 2)))
    
    # Dropout
    model.add(Dropout(rate=dropout_rate))     
    # fourth series
    model.add(Flatten())         
    
    # MLP, ouput layer
    model.add(Dense(classes,activation='softmax'))
    
    # Compile model
    model.compile( 
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy'],
        )    
    return model



In [ ]:
## Manually test function hyperparameters
#size = 128
#epoc = 50
#
#cnn = create_cnn_model()
#
#cnn.compile(
#  optimizer=optimizer,
#  loss='categorical_crossentropy',  
#  metrics=['accuracy'],
#)
#
#cnn.summary()
#
#cnn.fit(training_data_X,
#                  training_data_Y,
#                  validation_data=(testing_data_X,testing_data_Y),
#                  batch_size=size,
#                  epochs=epoc, 
#                  verbose=1, 
#                  callbacks=[PlotLossesKeras()])

In [131]:
# Hyper parameters tunning

model = KerasClassifier(build_fn=create_cnn_model, verbose=1)
n_cv = 2

batch_size_opt= [128] 
epochs_opt= [2]
optimizers_opt = ['adam', 'rmsprop','adamax']
dropout_opt = [0.1,0.25,0.35,0.5]
conv_activation_opt = ['relu','sigmoid']
pool_type_opt  = ['max', 'average']

parameters_opt = {
                  'pool_type': pool_type_opt,
                  'conv_activation': conv_activation_opt,    
                  'dropout_rate' : dropout_opt,
                  'optimizer': optimizers_opt,
                  'batch_size': batch_size_opt, 
                  'epochs': epochs_opt,      
                 }

grid_search = GridSearchCV(estimator=model, 
                           param_grid =parameters_opt, 
                           n_jobs=-1,
                           cv = n_cv)




In [ ]:
grid_result = grid_search.fit(training_data_X, training_data_Y)

In [ ]:
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_
print(best_parameters)
print(best_accuracy)

### BIBLIOGRAFÍA

https://www.kaggle.com/joparga3/2-tuning-parameters-for-logistic-regression

https://www.projectpro.io/recipes/optimize-hyper-parameters-of-logistic-regression-model-using-grid-search-in-python

https://github.com/harshitrai17152/Labeled-Faces-in-the-Wild.git
    